In [8]:
import pandas as pd
import arviz as az
from function_seasonality import *

In [9]:
train=pd.read_excel(r"../data/chem_train.xlsx")
test=pd.read_excel(r"../data/chem_test.xlsx")


In [10]:
train=make_periodic(train) 
test=make_periodic(test)

In [11]:
X_no2,y_no2=make_Xy(train,["cos_theta","sin_theta"],"no2") 
nuts_kernel,gpr_no2=model(X_no2,y_no2) 
nuts_kernel,gpr_no2,mcmc=train_model(X_no2,nuts_kernel,gpr_no2) 
torch.save(gpr_no2, "../models/no2_seasonality");

Sample: 100%|██████████████████████████████████████| 15000/15000 [06:43, 37.20it/s, step size=7.10e-01, acc. prob=0.931]
/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


In [12]:
# Define the model 
X_nox,y_no2=make_Xy(train,["cos_theta","sin_theta"],"nox") 
nuts_kernel_nox,gpr_nox=model(X_nox,y_no2) 
nuts_kernel_nox,gpr_nox,mcmc_nox=train_model(X_nox,nuts_kernel_nox,gpr_nox) 
torch.save(gpr_nox, "../models/nox_seasonality");

Sample: 100%|██████████████████████████████████████| 15000/15000 [05:20, 46.73it/s, step size=8.15e-01, acc. prob=0.912]
/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


In [7]:
gpr_no2

GPRegression(
  (kernel): RBF()
)